## Neural network for pax prediction

In [1]:
import tensorflow as tf
print(tf.__version__)

2.19.0


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('Datasets/augmented_SRQ_data_v3.csv')
df = df.drop(columns='Unnamed: 0')
df

Date  Flight Destination Airport Airline  Boarded  Day_of_Week  \
0      2023-09-14  AA2679                 DFW      AA      130            3   
1      2023-09-14  G41150                 PIT      G4      155            3   
2      2023-09-14   G4183                 ABE      G4      152            3   
3      2023-09-14   G4218                 AVL      G4      144            3   
4      2023-09-14  G42977                 CAK      G4      143            3   
...           ...     ...                 ...     ...      ...          ...   
12555  2025-07-20  WN4409                 IND      WN       71            6   
12556  2025-07-20  WN4605                 MDW      WN      125            6   
12557  2025-07-20  WN4606                 STL      WN       75            6   
12558  2025-07-20   XP306                 HVN      XP      132            6   
12559  2025-07-21  G41875                 AVL      G4      113            0   

       Month  Day_of_Month  Destination Airport_encoded  Airline_encoded  \
0          9            14                           23                0   
1          9            14                           60                5   
2          9            14                            0                5   
3          9            14                            6                5   
4          9            14                           14                5   
...      ...           ...                          ...              ...   
12555      7            20                           40                9   
12556      7            20                           49                9   
12557      7            20                           71                9   
12558      7            20                           36               10   
12559      7            21                            6                5   

        day_sin   day_cos     month_sin  month_cos   dow_sin   dow_cos  \
0      0.299363 -0.954139 -8.660254e-01       -0.5  0.433884 -0.900969   
1      0.299363 -0.954139 -8.660254e-01       -0.5  0.433884 -0.900969   
2      0.299363 -0.954139 -8.660254e-01       -0.5  0.433884 -0.900969   
3      0.299363 -0.954139 -8.660254e-01       -0.5  0.433884 -0.900969   
4      0.299363 -0.954139 -8.660254e-01       -0.5  0.433884 -0.900969   
...         ...       ...           ...        ...       ...       ...   
12555 -0.790776 -0.612106  1.224647e-16       -1.0 -0.781831  0.623490   
12556 -0.790776 -0.612106  1.224647e-16       -1.0 -0.781831  0.623490   
12557 -0.790776 -0.612106  1.224647e-16       -1.0 -0.781831  0.623490   
12558 -0.790776 -0.612106  1.224647e-16       -1.0 -0.781831  0.623490   
12559 -0.897805 -0.440394  1.224647e-16       -1.0  0.000000  1.000000   

       route_mean  route_median  route_std  
0       96.518750          99.5  51.749150  
1      148.430034         162.0  34.409989  
2      150.730769         157.0  23.763546  
3      126.549550         133.5  33.820457  
4      152.762452         160.0  22.801929  
...           ...           ...        ...  
12555  105.357143         111.0  35.060103  
12556  120.400000         122.0  39.266401  
12557  119.448980         128.0  35.404741  
12558  121.601604         129.0  37.176401  
12559  126.549550         133.5  33.820457  

[12560 rows x 19 columns]

In [4]:
plt.figure(figsize=(20,8))
plt.plot(df['Date'], df['Boarded'], marker='o', linestyle='-')
plt.title('Passenger Count Over Time')
plt.xlabel('Date')
plt.ylabel('Passenger Count')
plt.grid(True)
plt.tight_layout()
plt.show()

In [5]:
feature_columns = [  'Destination Airport_encoded', 'Airline_encoded',
    #'Day_of_Week', 'Month', 'Day_of_Month', 
    'day_sin',  'month_sin', 'dow_sin',
    'day_cos', 'month_cos', 'dow_cos', 
    'route_mean', 'route_median', 'route_std',]

In [8]:

X = df[feature_columns]
y = df['Boarded']  # or whatever your target column is
dates = pd.to_datetime(df['Date'])


split_index = int(len(df) * 0.8)

# Split data chronologically
X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]
dates_train = dates.iloc[:split_index]
dates_test = dates.iloc[split_index:]

print(f"Training set size: {X_train.shape[0]}")
print(f"Testing set size: {X_test.shape[0]}")
print(f"Training target range: {y_train.min()} to {y_train.max()}")
print(f"Testing target range: {y_test.min()} to {y_test.max()}")

Training set size: 10048
Testing set size: 2512
Training target range: 10 to 249
Testing target range: 10 to 238


In [9]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Log transform targets
y_train_log = np.log1p(y_train)
y_test_log = np.log1p(y_test)

In [10]:
# Get the split index (number of training samples)
split_index = len(y_train)

# Extract date and passenger count
df_timestamps = df['Date']
passenger_counts = df['Boarded']

# Plot
plt.figure(figsize=(20, 8))

plt.plot(df_timestamps[:split_index], passenger_counts[:split_index], label='Train', color='blue')
plt.plot(df_timestamps[split_index:], passenger_counts[split_index:], label='Test', color='orange')

plt.title('Passenger Count Over Time (Train/Test Split)', fontsize=16)
plt.xlabel('Flight Date', fontsize=14)
plt.ylabel('Passenger Count', fontsize=14)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [11]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(11,)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae', 'mse']
             )


C:\Users\KamarioSinclair\.conda\envs\pax-prediction\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
# For validation during training, use chronological split within training data
# Take last 20% of training data as validation set
val_split_index = int(len(X_train) * 0.8)

X_train_final = X_train_scaled[:val_split_index]
X_val = X_train_scaled[val_split_index:]
y_train_final = y_train_log.iloc[:val_split_index]
y_val = y_train_log.iloc[val_split_index:]

history = model.fit(X_train_final, y_train_final,
                    epochs=200,
                    batch_size=32,
                    validation_data=(X_val, y_val)  # or use x_test/y_test for validation if you prefer
                    verbose=1)

Epoch 1/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step - loss: 4.1212 - mae: 1.4395 - mse: 4.1212 - val_loss: 0.2039 - val_mae: 0.3602 - val_mse: 0.2039
Epoch 2/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3434 - mae: 0.4406 - mse: 0.3434 - val_loss: 0.2022 - val_mae: 0.3571 - val_mse: 0.2022
Epoch 3/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3406 - mae: 0.4379 - mse: 0.3406 - val_loss: 0.1897 - val_mae: 0.3290 - val_mse: 0.1897
Epoch 4/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3348 - mae: 0.4354 - mse: 0.3348 - val_loss: 0.2024 - val_mae: 0.3575 - val_mse: 0.2024
Epoch 5/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3244 - mae: 0.4291 - mse: 0.3244 - val_loss: 0.1870 - val_mae: 0.3205 - val_mse: 0.1870
Epoch 6/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.3212 - mae: 0.4251 - mse: 0.3212 - val_loss: 0.1861 - val_mae: 0.3171 - val_mse: 0.1861
Epoch 7/200
252/252 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.3206 - mae: 0.4230 - mse: 0.320

In [14]:
test_loss, test_mae, test_mse = model.evaluate(X_test_scaled, y_test_log, verbose=1)
print(f"Test MSE: {test_mse}, Test MAE: {test_mae}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1888 - mae: 0.2915 - mse: 0.1888 
Test MSE: 0.18882718682289124, Test MAE: 0.2914867103099823


In [15]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(10048, 11) (10048,)
(2512, 11) (2512,)


In [16]:
# Generate predictions
y_pred_log = model.predict(X_test_scaled)
y_pred = np.expm1(y_pred_log).flatten()
y_true = np.expm1(y_test_log).values

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step   


In [17]:
# Plot predictions vs actual - now dates align correctly
plt.figure(figsize=(15, 8))

# Plot first 100 test samples
n_samples = min(100, len(y_test))
dates_plot = dates_test.iloc[:n_samples]
y_true_plot = y_true[:n_samples]
y_pred_plot = y_pred[:n_samples]

plt.subplot(2, 1, 1)
plt.plot(dates_plot, y_true_plot, label='True', marker='o', alpha=0.7, markersize=4)
plt.plot(dates_plot, y_pred_plot, label='Predicted', marker='x', alpha=0.7, markersize=4)
plt.title('First 100 Test Samples - Chronological Order')
plt.xlabel('Date')
plt.ylabel('Passenger Count')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)


(array([20201. , 20201.5, 20202. , 20202.5, 20203. , 20203.5, 20204. ]),
 [Text(20201.0, 0, '04-23 00'),
  Text(20201.5, 0, '04-23 12'),
  Text(20202.0, 0, '04-24 00'),
  Text(20202.5, 0, '04-24 12'),
  Text(20203.0, 0, '04-25 00'),
  Text(20203.5, 0, '04-25 12'),
  Text(20204.0, 0, '04-26 00')])

In [18]:

# First 100 samples
y_true_100 = y_true[:100]
y_pred_100 = y_pred[:100].flatten()

# Scatter plot: True vs Predicted
plt.figure(figsize=(8, 8))
plt.scatter(y_true_plot, y_pred_plot, color='blue', alpha=0.7)
plt.plot([y_true_plot.min(), y_true_plot.max()],
         [y_true_plot.min(), y_true_plot.max()],
         color='red', linestyle='--', label='Perfect Prediction Line')

plt.title('Scatter Plot: True vs Predicted Values (First 100 Samples)')
plt.xlabel('True Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [19]:
# Plot training history
plt.subplot(2, 1, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Training History')
plt.xlabel('Epoch')
plt.ylabel('Loss (MSE)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [20]:
# Check your data preprocessing first
print("Data diagnostics:")
print(f"X_train shape: {X_train_scaled.shape}")
print(f"Feature statistics after scaling:")
print(f"Mean: {X_train_scaled.mean(axis=0)}")
print(f"Std: {X_train_scaled.std(axis=0)}")
print(f"Y_train_log statistics:")
print(f"Mean: {y_train_log.mean():.3f}, Std: {y_train_log.std():.3f}")
print(f"Min: {y_train_log.min():.3f}, Max: {y_train_log.max():.3f}")

Data diagnostics:
X_train shape: (10048, 11)
Feature statistics after scaling:
Mean: [ 2.82859369e-18  5.65718739e-18  6.78862486e-17  0.00000000e+00
 -9.90007793e-18  1.41429685e-17  5.09146865e-17 -8.48578108e-18
  2.40430464e-16 -1.13143748e-17             nan]
Std: [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. nan]
Y_train_log statistics:
Mean: 4.736, Std: 0.485
Min: 2.398, Max: 5.521
